In [4]:
import os
import dotenv
from pathlib import Path

from langchain_core.messages import AIMessage, HumanMessage
from langchain_community.document_loaders.text import TextLoader
from langchain_community.document_loaders import (
    WebBaseLoader, 
    PyPDFLoader, 
    Docx2txtLoader,
)
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

os.environ["USER_AGENT"] = "MyFastAPIApp/1.0"
dotenv.load_dotenv()

True

In [5]:
# Load docs

doc_paths = [
    "docs/test_rag.pdf"
]

docs = [] 
for doc_file in doc_paths:
    file_path = Path(doc_file)

    try:
        if doc_file.endswith(".pdf"):
            loader = PyPDFLoader(file_path)
        elif doc_file.endswith(".docx"):
            loader = Docx2txtLoader(file_path)
        elif doc_file.endswith(".txt") or doc_file.name.endswith(".md"):
            loader = TextLoader(file_path)
        else:
            print(f"Document type {doc_file.type} not supported.")
            continue

        docs.extend(loader.load())

    except Exception as e:
        print(f"Error loading document {doc_file.name}: {e}")


# Load URLs

url = "https://docs.streamlit.io/develop/quick-reference/release-notes"
try:
    loader = WebBaseLoader(url)
    docs.extend(loader.load())

except Exception as e:
    print(f"Error loading document from {url}: {e}")

In [6]:
docs

[Document(metadata={'source': 'docs\\test_rag.pdf', 'page': 0}, page_content='SAMPLE BUSINESS \nREQUIREMENTS \nDOCUMENT T EMPLATE \nSAMPLE BUSINESS REQUIREMENTS DOCUMENT TEMPLATE  \nVERSION HISTORY  \nVERSION  APPROVED BY  REVISION \nDATE  DESCRIPTION OF CHANGE  AUTHOR  \nNOTE TO USER: Overwrite the sample text included in this template to complete your project’s business \nrequirements document.'),
 Document(metadata={'source': 'docs\\test_rag.pdf', 'page': 1}, page_content='SAMPLE BUSINESS REQUIREMENTS DOCUMENT TEMPLATE T\nABLE OF CONTENTS  \n1.EXECUTIVE SUMMARY SNAPSHOT 3 \n2.PROJECT DESCRIPTION 4 \n3.PROJECT SCOPE 5 \n4.BUSINESS DRIVERS 6 \n5.CURRENT PROCESS 7 \n6.PROPOSED PROCESS 8 \n7.FUNCTIONAL REQUIREMENTS 9 \nPRIORITY 10 \nREQUIREMENTS CATEGORIES (RC1) 10 \n8. NON- FUNCTIONAL REQUIREMENTS 11 \n9.FINANCIAL STATEMENTS 12 \n10. COST AND BENEFIT 13 \n11. RESOURCES 14 \n12. SCHEDULE, TIMELINE, AND DEADLINES 15 \n13. ASSUMPTIONS 16 \n14. GLOSSARY 17 \n15. REFERENCES 18 \n16. APPENDI

In [7]:
# Split docs

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=5000,
    chunk_overlap=1000,
)

document_chunks = text_splitter.split_documents(docs)
document_chunks

[Document(metadata={'source': 'docs\\test_rag.pdf', 'page': 0}, page_content='SAMPLE BUSINESS \nREQUIREMENTS \nDOCUMENT T EMPLATE \nSAMPLE BUSINESS REQUIREMENTS DOCUMENT TEMPLATE  \nVERSION HISTORY  \nVERSION  APPROVED BY  REVISION \nDATE  DESCRIPTION OF CHANGE  AUTHOR  \nNOTE TO USER: Overwrite the sample text included in this template to complete your project’s business \nrequirements document.'),
 Document(metadata={'source': 'docs\\test_rag.pdf', 'page': 1}, page_content='SAMPLE BUSINESS REQUIREMENTS DOCUMENT TEMPLATE T\nABLE OF CONTENTS  \n1.EXECUTIVE SUMMARY SNAPSHOT 3 \n2.PROJECT DESCRIPTION 4 \n3.PROJECT SCOPE 5 \n4.BUSINESS DRIVERS 6 \n5.CURRENT PROCESS 7 \n6.PROPOSED PROCESS 8 \n7.FUNCTIONAL REQUIREMENTS 9 \nPRIORITY 10 \nREQUIREMENTS CATEGORIES (RC1) 10 \n8. NON- FUNCTIONAL REQUIREMENTS 11 \n9.FINANCIAL STATEMENTS 12 \n10. COST AND BENEFIT 13 \n11. RESOURCES 14 \n12. SCHEDULE, TIMELINE, AND DEADLINES 15 \n13. ASSUMPTIONS 16 \n14. GLOSSARY 17 \n15. REFERENCES 18 \n16. APPENDI

In [8]:
# Tokenize and load the documents to the vector store

vector_db = Chroma.from_documents(
    documents=document_chunks,
    embedding=OpenAIEmbeddings(),
)

In [9]:
vector_db

In [10]:
# Retrieve

def _get_context_retriever_chain(vector_db, llm):
    retriever = vector_db.as_retriever()
    prompt = ChatPromptTemplate.from_messages([
        MessagesPlaceholder(variable_name="messages"),
        ("user", "{input}"),
        ("user", "Given the option/input chosen by the user, mention if the option is correct or not. Inboth cases, give a short explianation focussing on most recent question."),
    ])
    retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

    return retriever_chain

In [11]:
def get_conversational_rag_chain(llm):
    retriever_chain = _get_context_retriever_chain(vector_db, llm)

    prompt = ChatPromptTemplate.from_messages([
        ("system",
        """You are a quiz master that ask questions to user. you will ask user a question and give 4 options. only one opion will be correct.
        You will have some context to help with your asking the questions and deciding the correct option, but now always would be completely related or helpful.
        You can also use your knowledge to assist answering the user's queries.\n
        {context}"""),
        MessagesPlaceholder(variable_name="messages"),
        ("user", "{input}"),
    ])
    stuff_documents_chain = create_stuff_documents_chain(llm, prompt)

    return create_retrieval_chain(retriever_chain, stuff_documents_chain)

In [12]:
# Augmented Generation

llm_stream_openai = ChatOpenAI(
    model="gpt-4o",  # Here you could use "o1-preview" or "o1-mini" if you already have access to them
    temperature=0.3,
    streaming=True,
)

llm_stream_anthropic = ChatAnthropic(
    model="claude-3-5-sonnet-20240620",
    temperature=0.3,
    streaming=True,
)

llm_stream = llm_stream_openai  # Select between OpenAI and Anthropic models for the response

messages = [
    {"role": "user", "content": "Hi"},
    {"role": "assistant", "content": "Hi there! How can I assist you today?"},
    {"role": "user", "content": "What is the latest version of Streamlit?"},
]
messages = [HumanMessage(content=m["content"]) if m["role"] == "user" else AIMessage(content=m["content"]) for m in messages]

conversation_rag_chain = get_conversational_rag_chain(llm_stream)
response_message = "*(RAG Response)*\n"
for chunk in conversation_rag_chain.pick("answer").stream({"messages": messages[:-1], "input": messages[-1].content}):
    response_message += chunk
    print(chunk, end="", flush=True)

messages.append({"role": "assistant", "content": response_message})

The latest version of Streamlit is 1.42.0, released on February 4, 2025.